In [1]:
import pandas as pd 
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

2024-06-15 15:24:48.400366: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 15:24:48.400479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 15:24:48.534394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
test_data = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
test_data

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [3]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/trained-deberta-xsmall")
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/trained-deberta-xsmall")

In [4]:
# Tokenize the test data
test_encodings = tokenizer(test_data['full_text'].tolist(), truncation=True, padding=True, max_length=1536)

In [5]:
# Create the test dataset
test_dataset = Dataset.from_dict(test_encodings)
test_dataset = test_dataset.add_column("essay_id", test_data['essay_id'].tolist())

In [6]:
predict_args = TrainingArguments(
    ".",
    per_device_eval_batch_size=4,
    report_to="none"
)

In [7]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=predict_args,
    tokenizer=tokenizer
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:
# Make predictions
predictions = trainer.predict(test_dataset)

In [9]:
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), dim=-1).numpy()

In [10]:
submission = pd.DataFrame({
    'essay_id': test_data['essay_id'],
    'score': predicted_labels.astype('int32')
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

In [11]:
submission

,essay_id,score
0,000d118,2
1,000fe60,2
2,001ab80,2
